<a href="https://colab.research.google.com/github/oraziotorre/MomentumShiftAI/blob/main/TennisDataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# Pandas per la gestione dei dati
import pandas as pd
import requests

from collections import Counter
from collections import defaultdict

# Per visualizzare i DataFrame su Google Colab in formato tabellare
from google.colab import data_table
data_table.enable_dataframe_formatter()

## Data Ingestion

Importo i csv grezzi dal MatchChartingProject su GitHub

In [32]:
url = "https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points-2020s.csv"
response = requests.get(url)

with open("charting-m-points-2020s.csv", "w") as f:
    f.write(response.text)

dataset1 = pd.read_csv("charting-m-points-2020s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points-2010s.csv"
response = requests.get(url)

with open("charting-m-points-2010s.csv", "w") as f:
    f.write(response.text)

dataset2 = pd.read_csv("charting-m-points-2010s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points-to-2009.csv"
response = requests.get(url)

with open("charting-m-points-to-2009.csv", "w") as f:
    f.write(response.text)

dataset3 = pd.read_csv("charting-m-points-to-2009.csv")


dataset = pd.concat([dataset1, dataset2, dataset3], ignore_index=True)

dataset

<ipython-input-32-ee841c57e572>:7: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset1 = pd.read_csv("charting-m-points-2020s.csv")


,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,Svr,1st,2nd,Notes,PtWinner
0,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,1,0,0,0.0,0.0,0-0,1.0,True,1,6s17y3w@,NaN,192,2
1,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,2,0,0,0.0,0.0,0-15,1.0,True,1,6w,6f29f2f2f3b3f2f3r2d#,159,2
2,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,3,0,0,0.0,0.0,0-30,1.0,True,1,4#,NaN,189,1
3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,4,0,0,0.0,0.0,15-30,1.0,True,1,4n,4b39f3b2f1f2f1f1b3b1b1f1b1f1s2s-1b1v3f2v2m2d@,158,2
4,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,5,0,0,0.0,0.0,15-40,1.0,True,1,6n,5f28f3b1s3f1d@,156,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022099,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4.0,3.0,0-40,46.0,False,2,R,NaN,NaN,1
1022100,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5.0,3.0,0-0,47.0,False,1,S,NaN,NaN,1
1022101,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5.0,3.0,15-0,47.0,False,1,S,NaN,NaN,1
1022102,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5.0,3.0,30-0,47.0,False,1,S,NaN,NaN,1


### Data Ingestion del ranking dei giocatori

In [33]:
url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_players.csv"
response = requests.get(url)

with open("atp_players.csv", "w") as f:
    f.write(response.text)

playerset = pd.read_csv("atp_players.csv")

# Sostituisci NaN con stringhe vuote prima della concatenazione
first = playerset['name_first'].fillna('').str.replace(' ', '_', regex=False).str.replace('-', '_', regex=False)
last = playerset['name_last'].fillna('').str.replace(' ', '_', regex=False).str.replace('-', '_', regex=False)

# Costruisci il full_name in modo sicuro
playerset['full_name'] = (
    first + '_' + last
).str.strip('_')  # Rimuove eventuali underscore iniziali/finali se una parte è vuota
playerset['full_name'] = playerset['full_name'].str.lower()
playerset

<ipython-input-33-48ffade97733>:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  playerset = pd.read_csv("atp_players.csv")


,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id,full_name
0,100001,Gardnar,Mulloy,R,19131122.0,USA,185.0,Q54544,gardnar_mulloy
1,100002,Pancho,Segura,R,19210620.0,ECU,168.0,Q54581,pancho_segura
2,100003,Frank,Sedgman,R,19271002.0,AUS,180.0,Q962049,frank_sedgman
3,100004,Giuseppe,Merlo,R,19271011.0,ITA,NaN,Q1258752,giuseppe_merlo
4,100005,Richard,Gonzalez,R,19280509.0,USA,188.0,Q53554,richard_gonzalez
...,...,...,...,...,...,...,...,...,...
65984,213700,Matvei,Kobiakov,U,NaN,RUS,NaN,NaN,matvei_kobiakov
65985,213701,Tobia Costanzo,Baragiola Mordini,U,NaN,ITA,NaN,NaN,tobia_costanzo_baragiola_mordini
65986,213702,Dominik,Wijntjes,U,NaN,NZL,NaN,NaN,dominik_wijntjes
65987,213703,Sam,Wensley,U,NaN,AUS,NaN,NaN,sam_wensley


Prendo da GitHub i rankings degli ultimi 20 anni dei giocatori di tennis

In [34]:
url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_current.csv"
response = requests.get(url)

with open("atp_rankings_current.csv", "w") as f:
    f.write(response.text)

ranking1 = pd.read_csv("atp_rankings_current.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_20s.csv"
response = requests.get(url)

with open("atp_rankings_20s.csv", "w") as f:
    f.write(response.text)

ranking2 = pd.read_csv("atp_rankings_20s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_10s.csv"
response = requests.get(url)

with open("atp_rankings_10s.csv", "w") as f:
    f.write(response.text)

ranking3 = pd.read_csv("atp_rankings_10s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_00s.csv"
response = requests.get(url)

with open("atp_rankings_00s.csv", "w") as f:
    f.write(response.text)

ranking4 = pd.read_csv("atp_rankings_00s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_90s.csv"
response = requests.get(url)

with open("atp_rankings_90s.csv", "w") as f:
    f.write(response.text)

ranking5 = pd.read_csv("atp_rankings_90s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_80s.csv"
response = requests.get(url)

with open("atp_rankings_80s.csv", "w") as f:
    f.write(response.text)

ranking6 = pd.read_csv("atp_rankings_80s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_70s.csv"
response = requests.get(url)

with open("atp_rankings_70s.csv", "w") as f:
    f.write(response.text)

ranking7 = pd.read_csv("atp_rankings_70s.csv")

rankingset = pd.concat([ranking1, ranking2, ranking3, ranking4, ranking5, ranking6, ranking7], ignore_index=True)

rankingset

,ranking_date,rank,player,points
0,20240101,1,104925,11245.0
1,20240101,2,207989,8855.0
2,20240101,3,106421,7600.0
3,20240101,4,206173,6490.0
4,20240101,5,126094,4805.0
...,...,...,...,...
3292944,19771212,95,100586,NaN
3292945,19771212,97,100363,NaN
3292946,19771212,98,100176,NaN
3292947,19771212,99,100233,NaN


Ottimizzazione e Correzione degli errori presenti nel dataset originale

In [35]:
# Crea il dizionario: full_name -> lista di player_id per ottimizzare l'esecuzione
player_id_map = defaultdict(list)

for _, row in playerset.iterrows():
  player_id_map[row['full_name']].append(row['player_id'])
  if(row['full_name']=='franko_skugor'):
    player_id_map['franco_skugor'].append(row['player_id'])
  if(row['full_name']=='jo_wilfried_tsonga'):
    player_id_map['jo_wilfred_tsonga'].append(row['player_id'])
  if(row['full_name']=='sam_groth'):
    player_id_map['samuel_groth'].append(row['player_id'])
  if(row['full_name']=='andres_artunedo_martinavarro'):
    player_id_map['andres_artunedo'].append(row['player_id'])
  if(row['full_name']=='marc_kevin_goellner'):
    player_id_map['mark_kevin_goellner'].append(row['player_id'])

rankingset['ranking_date'] = pd.to_datetime(rankingset['ranking_date'], format='%Y%m%d')

Creazione del dataset grezzo finale con i ranking dei giocatori

!!! Attenzione Tempo di Esecuzione minimo: 6 min

In [39]:
last_match = None
match_date = None
player_id_1 = None
player_id_2 = None
ranking_1 = None
ranking_2 = None

def extract_rankings(row):
    global last_match, match_date, player_id_1, player_id_2, ranking_1, ranking_2
    match_id = row['match_id']
    if match_id == last_match:
      return pd.Series([match_date.strftime("%Y-%m-%d"), player_id_1, player_id_2, ranking_1, ranking_2])
    else:
      last_match = match_id

      match_info = match_id.split('-')
      ranking_date = match_info[0]
      name1 = match_info[-2].lower()
      name2 = match_info[-1].lower()

      # Trova i player_id in modo sicuro
      player_id1 = player_id_map.get(name1)
      player_id2 = player_id_map.get(name2)

      # Converte la data di riferimento 'ranking_date' in formato datetime
      match_date = pd.to_datetime(ranking_date, format='%Y%m%d')

      for player_id in player_id1:
          try:
              filtered_ranking = rankingset[
                  (rankingset['player'] == player_id) &
                  (rankingset['ranking_date'] <= match_date)
              ]
              if not filtered_ranking.empty:
                  ranking_1 = filtered_ranking.sort_values(by='ranking_date', ascending=False).iloc[0]['rank']
                  player_id_1 = player_id
                  break  # Esce dal ciclo appena trova un ranking valido
          except Exception as e:
              continue  # Salta in caso di errore e prova con il prossimo ID

      for player_id in player_id2:
          try:
              filtered_ranking = rankingset[
                  (rankingset['player'] == player_id) &
                  (rankingset['ranking_date'] <= match_date)
              ]
              if not filtered_ranking.empty:
                  ranking_2 = filtered_ranking.sort_values(by='ranking_date', ascending=False).iloc[0]['rank']
                  player_id_2 = player_id
                  break
          except Exception as e:
              continue

      return pd.Series([match_date.strftime("%Y-%m-%d"), player_id_1, player_id_2, ranking_1, ranking_2])


dataset[['Date', 'Player1', 'Player2', 'Ranking1', 'Ranking2']] = dataset.apply(extract_rankings, axis=1)
dataset

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,Svr,1st,2nd,Notes,PtWinner,Date,Player1,Player2,Ranking1,Ranking2
0,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,1,0,0,0.0,0.0,0-0,1.0,True,1,6s17y3w@,NaN,192,2,2024-12-29,106329,100644,109,2
1,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,2,0,0,0.0,0.0,0-15,1.0,True,1,6w,6f29f2f2f3b3f2f3r2d#,159,2,2024-12-29,106329,100644,109,2
2,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,3,0,0,0.0,0.0,0-30,1.0,True,1,4#,NaN,189,1,2024-12-29,106329,100644,109,2
3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,4,0,0,0.0,0.0,15-30,1.0,True,1,4n,4b39f3b2f1f2f1f1b3b1b1f1b1f1s2s-1b1v3f2v2m2d@,158,2,2024-12-29,106329,100644,109,2
4,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,5,0,0,0.0,0.0,15-40,1.0,True,1,6n,5f28f3b1s3f1d@,156,1,2024-12-29,106329,100644,109,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022099,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4.0,3.0,0-40,46.0,False,2,R,NaN,NaN,1,1960-05-29,100113,100087,11,7
1022100,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5.0,3.0,0-0,47.0,False,1,S,NaN,NaN,1,1960-05-29,100113,100087,11,7
1022101,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5.0,3.0,15-0,47.0,False,1,S,NaN,NaN,1,1960-05-29,100113,100087,11,7
1022102,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5.0,3.0,30-0,47.0,False,1,S,NaN,NaN,1,1960-05-29,100113,100087,11,7


##Feature Engineering

### Controllo valori nulli e rari

In [40]:
print(dataset.isna().sum())

match_id         0
Pt               0
Set1             0
Set2             0
Gm1              1
Gm2              2
Pts              0
Gm#              1
TbSet            1
Svr              0
1st              0
2nd         637405
Notes       930137
PtWinner         0
Date             0
Player1          0
Player2          0
Ranking1         0
Ranking2         0
dtype: int64


In [41]:
dataset['Gm1'] = dataset['Gm1'].fillna(0).astype(int)
dataset['Gm2'] = dataset['Gm2'].fillna(0).astype(int)
dataset['Gm1'] = dataset['Gm1'].fillna(0)
dataset['TbSet'] = dataset['TbSet'].fillna
dataset = dataset.drop(columns=['Gm#', 'Notes', 'TbSet'], errors='ignore')
print(dataset.isna().sum())

match_id         0
Pt               0
Set1             0
Set2             0
Gm1              0
Gm2              0
Pts              0
Svr              0
1st              0
2nd         637405
PtWinner         0
Date             0
Player1          0
Player2          0
Ranking1         0
Ranking2         0
dtype: int64


In [42]:
def check_dataset(df, column_ranges=None):
    print("🔍 Controllo del dataset...\n")

    # 1. Valori nulli
    nulls = df.isnull().sum()
    nulls = nulls[nulls > 0]
    if not nulls.empty:
        print("⚠️  Colonne con valori nulli:")
        print(nulls)
    else:
        print("✅ Nessun valore nullo trovato.\n")

    # 2. Duplicati
    duplicate_rows = df.duplicated().sum()
    if duplicate_rows > 0:
        print(f"⚠️  Righe duplicate: {duplicate_rows}")
    else:
        print("✅ Nessuna riga duplicata.\n")

    # 3. Tipi di dato
    print("📊 Tipi di dato per colonna:")
    print(df.dtypes)
    print()

    # 4. Valori non numerici in colonne numeriche
    for col in df.select_dtypes(include=['object']).columns:
        try:
            pd.to_numeric(df[col])
            print(f"⚠️  La colonna '{col}' contiene stringhe ma potrebbe essere numerica.")
        except:
            pass

    # 5. Cardinalità sospetta
    print("\n🔢 Cardinalità delle colonne:")
    for col in df.columns:
        n_unique = df[col].nunique()
        if n_unique > 100 and df[col].dtype == 'object':
            print(f"⚠️  '{col}' ha {n_unique} valori unici (potrebbe contenere ID o errori)")
        else:
            print(f"{col}: {n_unique} valori unici")

    # 6. Range controlli se specificato
    if column_ranges:
        print("\n📏 Controllo range valori:")
        for col, (low, high) in column_ranges.items():
            if col in df.columns:
                out_of_bounds = df[(df[col] < low) | (df[col] > high)]
                if not out_of_bounds.empty:
                    print(f"⚠️  Colonna '{col}' ha {len(out_of_bounds)} valori fuori dal range [{low}, {high}]")
                else:
                    print(f"✅ Colonna '{col}' OK nel range [{low}, {high}]")

    print("\n✅ Controllo completato.")


check_dataset(dataset)

🔍 Controllo del dataset...

⚠️  Colonne con valori nulli:
2nd    637405
dtype: int64
⚠️  Righe duplicate: 1662
📊 Tipi di dato per colonna:
match_id    object
Pt           int64
Set1         int64
Set2         int64
Gm1          int64
Gm2          int64
Pts         object
Svr          int64
1st         object
2nd         object
PtWinner     int64
Date        object
Player1      int64
Player2      int64
Ranking1     int64
Ranking2     int64
dtype: object


🔢 Cardinalità delle colonne:
⚠️  'match_id' ha 6005 valori unici (potrebbe contenere ID o errori)
Pt: 980 valori unici
Set1: 4 valori unici
Set2: 4 valori unici
Gm1: 69 valori unici
Gm2: 70 valori unici
⚠️  'Pts' ha 144 valori unici (potrebbe contenere ID o errori)
Svr: 2 valori unici
⚠️  '1st' ha 274132 valori unici (potrebbe contenere ID o errori)
⚠️  '2nd' ha 214779 valori unici (potrebbe contenere ID o errori)
PtWinner: 2 valori unici
⚠️  'Date' ha 3626 valori unici (potrebbe contenere ID o errori)
Player1: 647 valori unici
Player2

Noto che ci sono valori non coerenti nelle colonne dei Set e dei Game. Decido di rimuovere tali partite

In [43]:
# Trova i match_id da escludere
invalid_match_ids = dataset.loc[dataset['Gm1'] > 60, 'match_id'].unique()

# Filtra via tutte le righe con quei match_id
dataset = dataset[~dataset['match_id'].isin(invalid_match_ids)]

In [44]:
# Trova i match_id da escludere (Set1 o Set2 > 2)
invalid_match_ids = dataset.loc[(dataset['Set1'] > 2) | (dataset['Set2'] > 2), 'match_id'].unique()

# Filtra via tutte le righe con quei match_id
dataset = dataset[~dataset['match_id'].isin(invalid_match_ids)]

Partite Rimosse per configurazione errata dei Set: 3

**Controllo la configurazione dei punteggi**

In [45]:
def normalize_pts_from_server(df):
    def flip_pts(pts):
        try:
            a, b = pts.split('-')
            return f"{b}-{a}"
        except:
            return pts  # se c'è un errore di parsing, restituisci il valore originale

    df['Pts'] = df.apply(
        lambda row: flip_pts(row['Pts']) if row['Svr'] == 2 else row['Pts'],
        axis=1
    )
    return df

dataset = normalize_pts_from_server(dataset)

In [46]:
conteggio_pts = dataset['Pts'].value_counts()

for simbolo, conteggio in conteggio_pts.items():
    print(f"'{simbolo}': {conteggio}")

'0-0': 163865
'15-0': 81042
'0-15': 79733
'15-15': 71815
'40-40': 67327
'30-15': 53791
'15-30': 52986
'30-30': 48470
'30-0': 45189
'0-30': 43770
'40-30': 39869
'30-40': 39812
'40-15': 39163
'15-40': 38469
'AD-40': 33929
'40-AD': 33309
'40-0': 27480
'0-40': 26501
'1-0': 1703
'1-1': 1665
'0-1': 1385
'1-2': 1240
'2-2': 1210
'2-1': 1120
'3-2': 1030
'3-3': 1002
'2-3': 951
'4-3': 867
'5-4': 855
'3-4': 845
'4-4': 842
'5-5': 811
'3-1': 774
'6-6': 760
'2-0': 756
'6-5': 743
'1-3': 741
'4-2': 737
'5-3': 728
'4-5': 707
'2-4': 698
'5-6': 690
'0-2': 667
'6-4': 659
'3-5': 652
'4-6': 568
'4-1': 518
'6-3': 498
'5-2': 496
'2-5': 480
'3-6': 462
'7-7': 443
'7-6': 412
'1-4': 391
'3-0': 367
'0-3': 361
'6-7': 356
'6-2': 332
'2-6': 305
'5-1': 302
'1-5': 249
'8-8': 242
'7-8': 233
'8-7': 217
'4-0': 193
'6-1': 177
'0-4': 170
'1-6': 169
'9-9': 146
'8-9': 130
'9-8': 121
'5-0': 108
'0-5': 90
'10-10': 79
'10-9': 74
'9-10': 72
'0-6': 50
'6-0': 49
'11-11': 48
'11-10': 42
'10-11': 37
'12-11': 25
'12-12': 24
'11-12': 23

In [47]:
def normalizza_vantaggi(p):
    esclusi = {"0", "15", "30", "40", "AD"}

    try:
        sx, dx = p.split('-')
    except ValueError:
        return p  # Formato non valido

    if sx in esclusi or dx in esclusi:
        return p  # È un punteggio classico del game, non tiebreak

    try:
        sx_int = int(sx)
        dx_int = int(dx)
    except ValueError:
        return p  # Uno dei due non è un numero (es. errore di dati)

    # Ora siamo sicuri che sono numeri da tiebreak
    if abs(sx_int - dx_int) <= 1 and sx_int>=6 and dx_int>=6 :
        if sx_int == dx_int:
            return '6-6'
        elif sx_int > dx_int:
            return 'AD-6'
        else:
            return '6-AD'
    else:
        return p

dataset['Pts'] = dataset['Pts'].apply(normalizza_vantaggi)
dataset

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Svr,1st,2nd,PtWinner,Date,Player1,Player2,Ranking1,Ranking2
0,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,1,0,0,0,0,0-0,1,6s17y3w@,NaN,2,2024-12-29,106329,100644,109,2
1,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,2,0,0,0,0,0-15,1,6w,6f29f2f2f3b3f2f3r2d#,2,2024-12-29,106329,100644,109,2
2,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,3,0,0,0,0,0-30,1,4#,NaN,1,2024-12-29,106329,100644,109,2
3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,4,0,0,0,0,15-30,1,4n,4b39f3b2f1f2f1f1b3b1b1f1b1f1s2s-1b1v3f2v2m2d@,2,2024-12-29,106329,100644,109,2
4,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,5,0,0,0,0,15-40,1,6n,5f28f3b1s3f1d@,1,2024-12-29,106329,100644,109,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022099,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4,3,40-0,2,R,NaN,1,1960-05-29,100113,100087,11,7
1022100,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5,3,0-0,1,S,NaN,1,1960-05-29,100113,100087,11,7
1022101,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5,3,15-0,1,S,NaN,1,1960-05-29,100113,100087,11,7
1022102,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5,3,30-0,1,S,NaN,1,1960-05-29,100113,100087,11,7


In [48]:
def ha_pts_oltre_il_limite(p):
    try:
        sx, dx = map(int, p.split('-'))
        return sx > 6 or dx > 6
    except:
        return False  # Non è nel formato x-y numerico

# Trova le righe sospette: Game1 == 6 e Game2 == 6, e Pts anomalo
righe_sospette = dataset[
    (dataset['Gm1'] == 6) &
    (dataset['Gm2'] == 6) &
    (dataset['Pts'].apply(ha_pts_oltre_il_limite))
]

# Trova tutti i match_id da escludere
invalid_match_ids = righe_sospette['match_id'].unique()

# Filtra via tutte le righe di quei match
dataset = dataset[~dataset['match_id'].isin(invalid_match_ids)]

In [49]:
invalid_match_ids = dataset[
    ((dataset['Gm1'] != 6) | (dataset['Gm2'] != 6)) &
    ((dataset['Pts'] == '1-0') | (dataset['Pts'] == '0-1'))
]['match_id'].unique()

# Elimina tutte le righe con quei match_id
dataset = dataset[~dataset['match_id'].isin(invalid_match_ids)]

Partite Rimosse causa configurazione errata dei punteggi: 111

### Numero di partite del dataset

In [50]:
simboli_finali = dataset['match_id'].nunique()
simboli_finali

5891

### Tipologie di esiti finali
Questi valori sono stati compararti con rispettivo significato presente sul MatchChartingProject

In [51]:
simboli_finali = Counter()


for index,row in dataset.iterrows():
  if(pd.isna(row['2nd'])):
    ultimo_carattere = row['1st'][-1]
    if(ultimo_carattere == ";"):
      ultimo_carattere = row['1st'][-2]
    simboli_finali[ultimo_carattere] += 1

  else:
    ultimo_carattere = row['2nd'][-1]
    if(ultimo_carattere == ";"):
      ultimo_carattere = row['2nd'][-2]
    simboli_finali[ultimo_carattere] += 1




for simbolo, conteggio in simboli_finali.most_common():
    print(f"'{simbolo}': {conteggio}")

'#': 327816
'*': 312350
'@': 303884
'n': 14941
'd': 14586
'S': 7247
'w': 3927
'R': 2171
'x': 874
'C': 298
'g': 95
'Q': 56
'!': 47
' ': 38
'P': 33
'e': 27
'3': 25
'1': 22
'9': 11
'8': 8
'2': 7
'+': 7
'7': 4
'=': 3
'^': 2
'b': 2
'c': 2
'4': 1
'&': 1
')': 1
'v': 1
'p': 1
'.': 1
'-': 1
'm': 1
'z': 1


#### Correzione dei casi rari e enfasi a informazioni importanti in un formato più gestibile

La descrizione in formato semplice, una riga per ogni stringa:


- `1` = `noinfo_1`: il giocatore 1 vince il punto, ma non è chiaro se è ace, errore, o altro.

- `2` = `noinfo_2`: il giocatore 2 vince il punto, ma non è chiaro se è ace, errore, o altro.

- `3` = `serve_ace_1`: punto diretto con il servizio (ace) fatto dal giocatore 1.

- `4` = `serve_ace_2`: punto diretto con il servizio (ace) fatto dal giocatore 2.

- `5` = `serve_miss2_1`: il giocatore 1 vince il punto su un errore di servizio del giocatore 2.

- `6` = `serve_miss1_2`: il giocatore 2 vince il punto su un errore di servizio del giocatore 1.

- `7` = `rally_winner_1`: il giocatore 1 vince il punto con un colpo vincente durante lo scambio.

- `8` = `rally_winner_2`: il giocatore 2 vince il punto con un colpo vincente durante lo scambio.

- `9` = `rally_forced2_1`: il giocatore 1 forza un errore del giocatore 2 durante lo scambio.

- `A` = `rally_forced1_2`: il giocatore 2 forza un errore del giocatore 1 durante lo scambio.

- `B` = `rally_unforced1_2`: il giocatore 2 vince per un errore non forzato del giocatore 1.

- `C` = `rally_unforced2_1`: il giocatore 1 vince per un errore non forzato del giocatore 2.

Attualmente non ho aggiunto considerazioni sul primo e secondo servizio, e su posizionamenti della palla e tipologia di colpi giocati.

Ho considerato le challenges Hawk-Eye perse come errori forzati.
Errori forzati e winners sono stati considerati come la stessa cosa nei servizi, si potrebbe fare la stessa considerazione anche nei rally

In [52]:
for index, row in dataset.iterrows():
    winner = row["PtWinner"]

    #On first serve points
    if pd.isna(row['2nd']):
        numeri = [c for c in row['1st'] if c.isdigit()]
        ultimo = row['1st'][-1]
        if(ultimo_carattere == ";"):
          ultimo_carattere = row['1st'][-2]

        #On serve points
        if len(numeri) == 1:
            if (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 1:
                dataset.at[index, 'PointType'] = '3'
            elif (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 2:
                dataset.at[index, 'PointType'] = '4'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 1:
                dataset.at[index, 'PointType'] = '5'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 2:
                dataset.at[index, 'PointType'] = '6'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                else:
                  dataset.at[index, 'PointType'] = '2'

        #On rally points
        else:
            if ultimo == '*' and winner == 1:
                dataset.at[index, 'PointType'] = '7'
            elif ultimo == '*' and winner == 2:
                dataset.at[index, 'PointType'] = '8'
            elif (ultimo == '#' or ultimo =="C") and winner == 1:
                dataset.at[index, 'PointType'] = '9'
            elif (ultimo == '#' or ultimo =="C") and winner == 2:
                dataset.at[index, 'PointType'] = 'A'
            elif ultimo == '@' and winner == 1:
                dataset.at[index, 'PointType'] = 'B'
            elif ultimo == '@' and winner == 2:
                dataset.at[index, 'PointType'] = 'C'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                elif winner == 2:
                  dataset.at[index, 'PointType'] = '2'

    #On second serve points
    else:
        numeri = [c for c in row['2nd'] if c.isdigit()]
        ultimo = row['2nd'][-1]
        if(ultimo_carattere == ";"):
          ultimo_carattere = row['2nd'][-2]

        #On serve points
        if len(numeri) == 1:
            if (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 1:
                dataset.at[index, 'PointType'] = '3'
            elif (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 2:
                dataset.at[index, 'PointType'] = '4'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 1:
                dataset.at[index, 'PointType'] = '5'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 2:
                dataset.at[index, 'PointType'] = '6'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                else:
                  dataset.at[index, 'PointType'] = '2'

        #On rally points
        else:
            if ultimo == '*' and winner == 1:
                dataset.at[index, 'PointType'] = '7'
            elif ultimo == '*' and winner == 2:
                dataset.at[index, 'PointType'] = '8'
            elif (ultimo == '#' or ultimo =="C") and winner == 1:
                dataset.at[index, 'PointType'] = '9'
            elif (ultimo == '#' or ultimo =="C") and winner == 2:
                dataset.at[index, 'PointType'] = 'A'
            elif ultimo == '@' and winner == 1:
                dataset.at[index, 'PointType'] = 'B'
            elif ultimo == '@' and winner == 2:
                dataset.at[index, 'PointType'] = 'C'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                elif winner == 2:
                  dataset.at[index, 'PointType'] = '2'


dataset

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Svr,1st,2nd,PtWinner,Date,Player1,Player2,Ranking1,Ranking2,PointType
0,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,1,0,0,0,0,0-0,1,6s17y3w@,NaN,2,2024-12-29,106329,100644,109,2,C
1,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,2,0,0,0,0,0-15,1,6w,6f29f2f2f3b3f2f3r2d#,2,2024-12-29,106329,100644,109,2,A
2,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,3,0,0,0,0,0-30,1,4#,NaN,1,2024-12-29,106329,100644,109,2,3
3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,4,0,0,0,0,15-30,1,4n,4b39f3b2f1f2f1f1b3b1b1f1b1f1s2s-1b1v3f2v2m2d@,2,2024-12-29,106329,100644,109,2,C
4,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,5,0,0,0,0,15-40,1,6n,5f28f3b1s3f1d@,1,2024-12-29,106329,100644,109,2,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022099,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4,3,40-0,2,R,NaN,1,1960-05-29,100113,100087,11,7,1
1022100,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5,3,0-0,1,S,NaN,1,1960-05-29,100113,100087,11,7,1
1022101,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5,3,15-0,1,S,NaN,1,1960-05-29,100113,100087,11,7,1
1022102,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5,3,30-0,1,S,NaN,1,1960-05-29,100113,100087,11,7,1


In [53]:
dataset = dataset.drop(columns=['1st', '2nd'], errors='ignore')

#### Controllo quantità tipologie di esiti finali

In [54]:
simboli_finali = dataset['PointType'].value_counts()

simboli_finali

,count
PointType,
C,151700
B,149338
9,145275
A,141342
7,116628
8,114512
3,63678
4,59029
6,18405


### Etichettatura

Creazione Etichette sulla vittoria del set corrente ed Etichetta sulla vittoria del match corrente.
Aggiunta di dati correlati al successivo sviluppo di modelli sulla predizione dei set

In [55]:
# 1. Crea identificatore univoco del set (combinazione match_id e somma set)
dataset['SetID'] = dataset['match_id'].astype(str) + '_' + (dataset['Set1'] + dataset['Set2']).astype(str)

# 2. Conta i punti all'interno di ciascun set
dataset['PtSet'] = dataset.groupby('SetID').cumcount() + 1

# 3. Calcola il vincitore di ciascun set (ultimo punto del set)
set_winners = dataset.groupby('SetID')['PtWinner'].last()
dataset['SetWinner'] = dataset['SetID'].map(set_winners)

# 4. Calcola il vincitore del match (ultimo punto del match)
match_winners = dataset.groupby('match_id')['PtWinner'].last()
dataset['MatchWinner'] = dataset['match_id'].map(match_winners)

dataset

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Svr,PtWinner,Date,Player1,Player2,Ranking1,Ranking2,PointType,SetID,PtSet,SetWinner,MatchWinner
0,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,1,0,0,0,0,0-0,1,2,2024-12-29,106329,100644,109,2,C,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,1,2,2
1,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,2,0,0,0,0,0-15,1,2,2024-12-29,106329,100644,109,2,A,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,2,2,2
2,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,3,0,0,0,0,0-30,1,1,2024-12-29,106329,100644,109,2,3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,3,2,2
3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,4,0,0,0,0,15-30,1,2,2024-12-29,106329,100644,109,2,C,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,4,2,2
4,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,5,0,0,0,0,15-40,1,1,2024-12-29,106329,100644,109,2,B,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,5,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022099,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4,3,40-0,2,1,1960-05-29,100113,100087,11,7,1,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,32,1,1
1022100,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5,3,0-0,1,1,1960-05-29,100113,100087,11,7,1,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,33,1,1
1022101,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5,3,15-0,1,1,1960-05-29,100113,100087,11,7,1,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,34,1,1
1022102,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5,3,30-0,1,1,1960-05-29,100113,100087,11,7,1,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,35,1,1


#### Analisi Bilanciamento Etichette

In [56]:
simboli_finali = dataset['SetWinner'].value_counts()

simboli_finali

,count
SetWinner,
1,507317
2,481175


In [57]:
simboli_finali = dataset['MatchWinner'].value_counts()

simboli_finali

,count
MatchWinner,
1,512235
2,476257


### Feature Derivate


- p1_win_nobreak_point: P1 vince un game point mentre serve (tiene il servizio)

- p1_win_break_point: P1 vince un break point mentre risponde

- p1_lost_nobreak_point: P1 perde un game point mentre serve

- p1_lost_break_point: P1 perde un break point mentre risponde

- p2_win_nobreak_point: P2 vince un game point mentre serve

- p2_win_break_point: P2 vince un break point mentre risponde

- p2_lost_nobreak_point: P2 perde un game point mentre serve

- p2_lost_break_point: P2 perde un break point mentre risponde


In [58]:
dataset['p1_win_nobreak_point'] = 0
dataset['p2_win_nobreak_point'] = 0
dataset['p1_win_break_point'] = 0
dataset['p2_win_break_point'] = 0
dataset['p1_lost_nobreak_point'] = 0
dataset['p2_lost_nobreak_point'] = 0
dataset['p1_lost_break_point'] = 0
dataset['p2_lost_break_point'] = 0

punteggi_6_x = [f'6-{i}' for i in range(6)]
punteggi_game_point = ['40-0', '40-15', '40-30', 'AD-40', 'AD-6'] + punteggi_6_x

punteggio_favorevole = dataset['Pts'].isin(punteggi_game_point)
p1_vittoria_punto = dataset['PtWinner'] == 1
servizio = dataset['Svr'] == 1

condizione = punteggio_favorevole & p1_vittoria_punto & servizio
dataset.loc[condizione, 'p1_win_nobreak_point'] = 1
condizione = punteggio_favorevole & p1_vittoria_punto & (~servizio)
dataset.loc[condizione, 'p1_win_break_point'] = 1
condizione = punteggio_favorevole & (~p1_vittoria_punto) & servizio
dataset.loc[condizione, 'p1_lost_nobreak_point'] = 1
condizione = punteggio_favorevole & (~p1_vittoria_punto) & (~servizio)
dataset.loc[condizione, 'p1_lost_break_point'] = 1


punteggi_6_x = [f'{i}-6' for i in range(6)]
punteggi_game_point = ['0-40', '15-40', '30-40', '40-AD', '6-AD'] + punteggi_6_x
punteggio_favorevole = dataset['Pts'].isin(punteggi_game_point)
p2_vittoria_punto = dataset['PtWinner'] == 2
servizio = dataset['Svr'] == 2

condizione = punteggio_favorevole & p2_vittoria_punto & servizio
dataset.loc[condizione, 'p2_win_nobreak_point'] = 1
condizione = punteggio_favorevole & p2_vittoria_punto & (~servizio)
dataset.loc[condizione, 'p2_win_break_point'] = 1
condizione = punteggio_favorevole & (~p2_vittoria_punto) & servizio
dataset.loc[condizione, 'p2_lost_nobreak_point'] = 1
condizione = punteggio_favorevole & (~p2_vittoria_punto) & (~servizio)
dataset.loc[condizione, 'p2_lost_break_point'] = 1

dataset

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Svr,PtWinner,Date,...,SetWinner,MatchWinner,p1_win_nobreak_point,p2_win_nobreak_point,p1_win_break_point,p2_win_break_point,p1_lost_nobreak_point,p2_lost_nobreak_point,p1_lost_break_point,p2_lost_break_point
0,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,1,0,0,0,0,0-0,1,2,2024-12-29,...,2,2,0,0,0,0,0,0,0,0
1,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,2,0,0,0,0,0-15,1,2,2024-12-29,...,2,2,0,0,0,0,0,0,0,0
2,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,3,0,0,0,0,0-30,1,1,2024-12-29,...,2,2,0,0,0,0,0,0,0,0
3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,4,0,0,0,0,15-30,1,2,2024-12-29,...,2,2,0,0,0,0,0,0,0,0
4,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,5,0,0,0,0,15-40,1,1,2024-12-29,...,2,2,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022099,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4,3,40-0,2,1,1960-05-29,...,1,1,0,0,1,0,0,0,0,0
1022100,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5,3,0-0,1,1,1960-05-29,...,1,1,0,0,0,0,0,0,0,0
1022101,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5,3,15-0,1,1,1960-05-29,...,1,1,0,0,0,0,0,0,0,0
1022102,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5,3,30-0,1,1,1960-05-29,...,1,1,0,0,0,0,0,0,0,0


Creazione delle feature derivate dalla precedente analisi del colpo con cui è finito lo scambio

In [59]:
mapping = {
    '3': 'serve_ace_1',
    '4': 'serve_ace_2',
    '5': 'serve_miss2_1',
    '6': 'serve_miss1_2',
    '7': 'rally_winner_1',
    '8': 'rally_winner_2',
    '9': 'rally_forced2_1',
    'A': 'rally_forced1_2',
    'B': 'rally_unforced1_2',
    'C': 'rally_unforced2_1'
}

# Per ogni simbolo nella mappa, crea una nuova colonna
for symbol, column in mapping.items():
    dataset[column] = (dataset['PointType'] == symbol).astype(int)

dataset

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Svr,PtWinner,Date,...,serve_ace_1,serve_ace_2,serve_miss2_1,serve_miss1_2,rally_winner_1,rally_winner_2,rally_forced2_1,rally_forced1_2,rally_unforced1_2,rally_unforced2_1
0,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,1,0,0,0,0,0-0,1,2,2024-12-29,...,0,0,0,0,0,0,0,0,0,1
1,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,2,0,0,0,0,0-15,1,2,2024-12-29,...,0,0,0,0,0,0,0,1,0,0
2,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,3,0,0,0,0,0-30,1,1,2024-12-29,...,1,0,0,0,0,0,0,0,0,0
3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,4,0,0,0,0,15-30,1,2,2024-12-29,...,0,0,0,0,0,0,0,0,0,1
4,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,5,0,0,0,0,15-40,1,1,2024-12-29,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022099,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4,3,40-0,2,1,1960-05-29,...,0,0,0,0,0,0,0,0,0,0
1022100,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5,3,0-0,1,1,1960-05-29,...,0,0,0,0,0,0,0,0,0,0
1022101,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5,3,15-0,1,1,1960-05-29,...,0,0,0,0,0,0,0,0,0,0
1022102,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5,3,30-0,1,1,1960-05-29,...,0,0,0,0,0,0,0,0,0,0


In [60]:
dataset= dataset.drop(columns=['Date'])

# Riposizionamento delle nuove colonne
is_final_index = dataset.columns.get_loc('Svr')
player1_column = dataset.pop('Player1')
player2_column = dataset.pop('Player2')
ranking1_column = dataset.pop('Ranking1')
ranking2_column = dataset.pop('Ranking2')

dataset.insert(is_final_index, 'Player1', player1_column)
dataset.insert(is_final_index + 1, 'Player2', player2_column)
dataset.insert(is_final_index + 2, 'Ranking1', ranking1_column)
dataset.insert(is_final_index + 3, 'Ranking2', ranking2_column)


dataset

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Player1,Player2,Ranking1,...,serve_ace_1,serve_ace_2,serve_miss2_1,serve_miss1_2,rally_winner_1,rally_winner_2,rally_forced2_1,rally_forced1_2,rally_unforced1_2,rally_unforced2_1
0,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,1,0,0,0,0,0-0,106329,100644,109,...,0,0,0,0,0,0,0,0,0,1
1,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,2,0,0,0,0,0-15,106329,100644,109,...,0,0,0,0,0,0,0,1,0,0
2,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,3,0,0,0,0,0-30,106329,100644,109,...,1,0,0,0,0,0,0,0,0,0
3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,4,0,0,0,0,15-30,106329,100644,109,...,0,0,0,0,0,0,0,0,0,1
4,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,5,0,0,0,0,15-40,106329,100644,109,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022099,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4,3,40-0,100113,100087,11,...,0,0,0,0,0,0,0,0,0,0
1022100,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5,3,0-0,100113,100087,11,...,0,0,0,0,0,0,0,0,0,0
1022101,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5,3,15-0,100113,100087,11,...,0,0,0,0,0,0,0,0,0,0
1022102,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5,3,30-0,100113,100087,11,...,0,0,0,0,0,0,0,0,0,0


## Salvataggio dataset

In [61]:
dataset.to_csv('tennis.csv')
dataset

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Player1,Player2,Ranking1,...,serve_ace_1,serve_ace_2,serve_miss2_1,serve_miss1_2,rally_winner_1,rally_winner_2,rally_forced2_1,rally_forced1_2,rally_unforced1_2,rally_unforced2_1
0,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,1,0,0,0,0,0-0,106329,100644,109,...,0,0,0,0,0,0,0,0,0,1
1,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,2,0,0,0,0,0-15,106329,100644,109,...,0,0,0,0,0,0,0,1,0,0
2,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,3,0,0,0,0,0-30,106329,100644,109,...,1,0,0,0,0,0,0,0,0,0
3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,4,0,0,0,0,15-30,106329,100644,109,...,0,0,0,0,0,0,0,0,0,1
4,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,5,0,0,0,0,15-40,106329,100644,109,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022099,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4,3,40-0,100113,100087,11,...,0,0,0,0,0,0,0,0,0,0
1022100,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5,3,0-0,100113,100087,11,...,0,0,0,0,0,0,0,0,0,0
1022101,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5,3,15-0,100113,100087,11,...,0,0,0,0,0,0,0,0,0,0
1022102,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5,3,30-0,100113,100087,11,...,0,0,0,0,0,0,0,0,0,0
